<div style="background-color:#000;"><img src="pqn.png"></img></div>

In [ ]:
```python
import riskfolio as rp
import pandas as pd
import yfinance as yf

In [ ]:
```

## Fetch historical stock data and calculate returns

In [ ]:
We will fetch historical stock price data for a list of assets from Yahoo Finance. This data will be used to calculate daily returns, which are necessary for constructing a portfolio.

In [ ]:
```python
assets = ["QQQ", "AAPL", "MSFT", "AMZN", "GOOGL", "META", "TSLA", "NVDA"]

In [ ]:
data = yf.download(assets, start="2016-01-01", end="2019-12-30")["Adj Close"]
data.index = pd.to_datetime(data.index)

In [ ]:
returns = data.pct_change().dropna()
bench_returns = returns.pop("QQQ").to_frame()

In [ ]:
```

In [ ]:
We define a list of asset symbols that we are interested in analyzing. These symbols include popular technology stocks and an ETF. Using the `yfinance` library, we download adjusted closing price data for these assets over a specified date range. The data is then converted to a pandas DataFrame with a datetime index for easier manipulation. We calculate daily percentage returns for the assets and store them in a DataFrame, dropping any missing values. The benchmark returns, specifically for the QQQ ETF, are also isolated for later comparison.

## Set up and configure the portfolio

In [ ]:
Next, we will set up a portfolio object to store the asset returns and configure it with desired statistical methods. We also define some constraints for the optimization process.

In [ ]:
```python
port = rp.Portfolio(returns=returns)
port.assets_stats(method_mu="hist", method_cov="hist", d=0.94)

In [ ]:
port.kindbench = False
port.benchindex = bench_returns

In [ ]:
port.allowTE = True
port.TE = 0.008

In [ ]:
```

In [ ]:
We create a portfolio object using the `riskfolio` library and load the calculated returns into it. We then calculate asset statistics like mean and covariance using historical data, setting the decay factor for exponential smoothing. We indicate that there are no predefined benchmark weights, choosing instead to use the benchmark index we previously defined. We enable the use of tracking error constraints, which will allow us to control how much the portfolio's returns can deviate from the benchmark. We set a maximum tracking error of 0.8%, ensuring our portfolio remains closely aligned with the benchmark.

## Optimize the portfolio and visualize the result

In [ ]:
We will perform the portfolio optimization using the specified risk model and objective function. After obtaining the optimal weights, we will visualize the portfolio composition with a pie chart.

In [ ]:
```python
w = port.optimization(
    model="Classic",
    rm="CVaR",
    obj="Sharpe",
    rf=0,
    l=0,
    hist=True
)

In [ ]:
ax = rp.plot_pie(
    w=w,
    title="Sharpe Mean CVaR",
    others=0.05,
    nrow=25,
    cmap="tab20",
    height=6,
    width=10,
    ax=None,
)

In [ ]:
```

In [ ]:
We perform the portfolio optimization using a classic mean-variance model. The risk measure chosen is Conditional Value-at-Risk (CVaR), which focuses on tail risk. We aim to maximize the Sharpe ratio, which balances risk and return. The optimization assumes a risk-free rate of 0% and uses historical data for calculations. The result is a set of optimal asset weights, which we then visualize using a pie chart. The pie chart helps us understand the composition of the portfolio, showing the relative weight of each asset. We also group smaller allocations under "others" for simplicity.

## Compare portfolio and benchmark returns

In [ ]:
We will calculate the cumulative returns for both the optimized portfolio and the benchmark. Finally, we will plot these returns to visually compare their performance over time.

In [ ]:
```python
portfolio_returns = returns.dot(w)

In [ ]:
comparison = pd.concat([portfolio_returns, bench_returns], axis=1)

In [ ]:
comparison.add(1).cumprod().plot(figsize=(10, 6), title="Cumulative Returns: QQQ vs Portfolio")

In [ ]:
```

In [ ]:
We calculate the portfolio's returns by multiplying the asset returns by their respective weights. This gives us a time series of portfolio returns. We then create a DataFrame that combines these portfolio returns with the benchmark returns, aligning them for comparison. We calculate cumulative returns by adding one to each return and taking the cumulative product over time. Finally, we plot these cumulative returns to visually compare the performance of the optimized portfolio against the QQQ benchmark. The resulting plot provides a clear view of how well the portfolio tracks the benchmark and highlights any periods of outperformance or underperformance.

## Your next steps

In [ ]:
Try adjusting the maximum allowed tracking error to see how it affects the portfolio's composition and returns. You can also explore changing the risk measure from CVaR to another option like variance or maximum drawdown. Experiment with different objective functions to see how they influence the optimization results. Each of these changes can provide insights into how different constraints and goals impact the behavior of a portfolio.
```

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.